In [2]:
pip install selenium pandas

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/?BD=7200&TP=107"
    driver.get(url)

    # Step 2: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 3: Wait for the first results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "post.yes"))
    )

    # Step 4: Scroll and Load More Results
    last_height = driver.execute_script("return document.body.scrollHeight")
    articles = []
    while True:
        # Find all articles
        new_articles = driver.find_elements(By.CSS_SELECTOR, "article.post.yes")
        articles.extend(new_articles)

        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Allow time for new content to load

        # Check if the page height has changed
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Step 5: Extract Information from Each Article
    results = []
    for article in articles:
        title = article.find_element(By.CSS_SELECTOR, "h3").text
        license_number = article.find_element(By.CSS_SELECTOR, "span[id^='lic']").text
        license_type = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Type:')]]").text
        license_status = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Status:')]]").text
        expiration_date = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Expiration Date:')]]").text
        city = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'City:')]]").text
        state = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'State:')]]").text
        county = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'County:')]]").text
        zip_code = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Zip:')]]").text

        results.append({
            "Title": title,
            "License Number": license_number,
            "License Type": license_type,
            "License Status": license_status,
            "Expiration Date": expiration_date,
            "City": city,
            "State": state,
            "County": county,
            "Zip": zip_code,
        })

    # Step 6: Save to a DataFrame
    df = pd.DataFrame(results)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacy_data.csv", index=False)

finally:
    # Close the browser
    driver.quit()


In [22]:
df.head()

In [23]:
# Second Attempt; Issue with the above is that it is not searching for specifically sterile compound pharmacies so we are getting the wrong info back and i have the wrong table format.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/?BD=7200&TP=107"
    driver.get(url)

    # Step 2: Wait for the License Type dropdown to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "licenseType"))
    )
    
    # Step 3: Select "Sterile Compounding Pharmacy" from the dropdown
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")
    
    # Step 4: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 5: Wait for the filtered results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "post.yes"))
    )

    # Step 6: Scroll and Load More Results
    last_height = driver.execute_script("return document.body.scrollHeight")
    articles = []
    while True:
        # Find all articles
        new_articles = driver.find_elements(By.CSS_SELECTOR, "article.post.yes")
        articles.extend(new_articles)

        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Allow time for new content to load

        # Check if the page height has changed
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Step 7: Extract Information from Each Article
    results = []
    for article in articles:
        title = article.find_element(By.XPATH, ".//li/h3").text
        license_number = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Number:')]]/a/span").text
        license_type = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Type:')]]").text.split(": ")[-1]
        license_status = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Status:')]]").text.split(": ")[-1]
        expiration_date = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Expiration Date:')]]").text.split(": ")[-1]
        city = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'City:')]]/span").text
        state = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'State:')]]/span").text
        county = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'County:')]]").text.split(": ")[-1]
        zip_code = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Zip:')]]").text.split(": ")[-1]

        results.append({
            "Title": title,
            "License Number": license_number,
            "License Type": license_type,
            "License Status": license_status,
            "Expiration Date": expiration_date,
            "City": city,
            "State": state,
            "County": county,
            "Zip": zip_code,
        })

    # Step 8: Save to a DataFrame
    df = pd.DataFrame(results)

    # Print or save the DataFrame
    print(df)
    df.to_csv("sterile_compounding_pharmacy_data.csv", index=False)

finally:
    # Close the browser
    driver.quit()


In [25]:
#This is a third attempt; I am now going to use the advanced search and try and limit to just active pharmacies to stop returning so many that are old and cancelled. 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the new webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the License Type dropdown to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Step 3: Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Step 4: Select "Sterile Compounding Pharmacy" from the License Type dropdown
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 5: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 6: Wait for the filtered results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "post.yes"))
    )

    # Step 7: Scroll and Load More Results
    last_height = driver.execute_script("return document.body.scrollHeight")
    articles = []
    while True:
        # Find all articles
        new_articles = driver.find_elements(By.CSS_SELECTOR, "article.post.yes")
        articles.extend(new_articles)

        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Allow time for new content to load

        # Check if the page height has changed
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Step 8: Extract Information from Each Article
    results = []
    for article in articles:
        title = article.find_element(By.XPATH, ".//li/h3").text
        license_number = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Number:')]]/a/span").text
        license_type = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Type:')]]").text.split(": ")[-1]
        license_status = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Status:')]]").text.split(": ")[-1]
        expiration_date = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Expiration Date:')]]").text.split(": ")[-1]
        city = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'City:')]]/span").text
        state = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'State:')]]/span").text
        county = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'County:')]]").text.split(": ")[-1]
        zip_code = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Zip:')]]").text.split(": ")[-1]

        results.append({
            "Title": title,
            "License Number": license_number,
            "License Type": license_type,
            "License Status": license_status,
            "Expiration Date": expiration_date,
            "City": city,
            "State": state,
            "County": county,
            "Zip": zip_code,
        })

    # Step 9: Save to a DataFrame
    df = pd.DataFrame(results)

    # Print or save the DataFrame
    print(df)
    df.to_csv("sterile_compounding_pharmacy_data_2.csv", index=False)

finally:
    # Close the browser
    driver.quit()


In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the new webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the License Type dropdown to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Step 3: Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Step 4: Select "Sterile Compounding Pharmacy" from the License Type dropdown
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 5: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 6: Wait for the filtered results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "post.yes"))
    )

    # Step 7: Scroll and Load More Results
    last_height = driver.execute_script("return document.body.scrollHeight")
    articles = []
    while True:
        # Find all articles with a class 'post' and a specific id
        article_elements = driver.find_elements(By.CSS_SELECTOR, "article.post")

        # Loop through each article and extract the data
        for article in article_elements:
            article_id = article.get_attribute("id")  # Extract article id

            title = article.find_element(By.XPATH, ".//li/h3").text

            # Wait until the License Number element is found (with the dynamic 'lic' id)
            try:
                license_number = WebDriverWait(article, 10).until(
                    EC.presence_of_element_located(
                        (By.XPATH, ".//li[strong[contains(text(), 'License Number:')]]/a/span[starts-with(@id, 'lic')]")
                    )
                ).text
            except:
                license_number = "Not Available"

            license_type = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Type:')]]").text.split(": ")[-1]
            license_status = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Status:')]]").text.split(": ")[-1]
            expiration_date = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Expiration Date:')]]").text.split(": ")[-1]
            city = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'City:')]]/span").text
            state = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'State:')]]/span").text
            county = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'County:')]]").text.split(": ")[-1]
            zip_code = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Zip:')]]").text.split(": ")[-1]

            # Add the data to the results list
            articles.append({
                "Article ID": article_id,
                "Title": title,
                "License Number": license_number,
                "License Type": license_type,
                "License Status": license_status,
                "Expiration Date": expiration_date,
                "City": city,
                "State": state,
                "County": county,
                "Zip": zip_code,
            })

        # Scroll down to load more results
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Allow time for new content to load

        # Check if the page height has changed (to detect if more articles are available)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Step 8: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Print or save the DataFrame
    print(df)
    df.to_csv("sterile_compounding_pharmacy_data.csv", index=False)

finally:
    # Close the browser
    driver.quit()


## Next Attempt 

In [29]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the new webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the License Type dropdown to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Step 3: Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Step 4: Select "Sterile Compounding Pharmacy" from the License Type dropdown
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 5: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 6: Wait for the filtered results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "post.yes"))
    )

    # Step 7: Scroll and Load More Results
    articles = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Find all articles with a numeric id
        article_elements = driver.find_elements(By.CSS_SELECTOR, "article.post")
        
        for article in article_elements:
            # Extract the numeric ID from the article's `id` attribute
            article_id = article.get_attribute("id")
            if not re.match(r'^\d+$', article_id):
                continue  # Skip non-numeric IDs
            
            try:
                # Extract details from the <li> elements within the article
                li_elements = article.find_elements(By.XPATH, ".//li")
                article_data = {"Article ID": article_id}
    
                for li in li_elements:
                    text = li.text.strip()
                    
                    if "License Number:" in text:
                        # Extract License Number
                        article_data["License Number"] = li.find_element(By.XPATH, ".//a/span[starts-with(@id, 'lic')]").text
                    
                    elif "License Type:" in text:
                        # Extract License Type
                        article_data["License Type"] = text.split(": ")[-1]
                    
                    elif "License Status:" in text:
                        # Extract License Status
                        article_data["License Status"] = text.split(": ")[-1]
                    
                    elif "Expiration Date:" in text:
                        # Extract Expiration Date
                        article_data["Expiration Date"] = text.split(": ")[-1]
                    
                    elif "City:" in text:
                        # Extract City
                        article_data["City"] = li.find_element(By.XPATH, ".//span").text
                    
                    elif "State:" in text:
                        # Extract State
                        article_data["State"] = li.find_element(By.XPATH, ".//span").text
                    
                    elif "County:" in text:
                        # Extract County
                        article_data["County"] = text.split(": ")[-1]
                    
                    elif "Zip:" in text:
                        # Extract Zip Code
                        article_data["Zip"] = text.split(": ")[-1]
                
                # Add the collected data to the articles list
                if article_data not in articles:  # Avoid duplicates
                    articles.append(article_data)
    
            except Exception as e:
                print(f"Error processing article ID {article_id}: {e}")
        
        # Scroll down to load more results
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Allow time for new content to load
    
        # Check if the page height has changed (to detect if more articles are available)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # Step 8: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Print or save the DataFrame
    print(df)
    df.to_csv("sterile_compounding_pharmacy_data.csv", index=False)

finally:
    # Close the browser
    driver.quit()


In [30]:
df.head()

## New Strategy:  Use UL when Class = "Actions" to locate LIs 

In [31]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the License Type dropdown to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Step 3: Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Step 4: Select "Sterile Compounding Pharmacy" from the License Type dropdown
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 5: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 6: Wait for the initial results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "actions"))
    )

    # Step 7: Scroll and load more results
    articles = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_pause_time = 2  # Adjust based on network speed

    while True:
        # Find all <ul> elements with class "actions"
        ul_elements = driver.find_elements(By.CSS_SELECTOR, "ul.actions")
        
        for ul in ul_elements:
            try:
                # Initialize a dictionary to store data
                article_data = {}

                # Extract the pharmacy name (h3 within the <ul>)
                try:
                    article_data["Pharmacy Name"] = ul.find_element(By.XPATH, "./li/h3").text
                except:
                    article_data["Pharmacy Name"] = "Not Available"

                # Extract data for each relevant field
                for li in ul.find_elements(By.XPATH, "./li"):
                    text = li.text.strip()

                    if "License Number:" in text:
                        article_data["License Number"] = li.find_element(By.XPATH, ".//a/span").text

                    elif "License Type:" in text:
                        article_data["License Type"] = text.split(": ")[-1]

                    elif "License Status:" in text:
                        article_data["License Status"] = text.split(": ")[-1]

                    elif "Expiration Date:" in text:
                        article_data["Expiration Date"] = text.split(": ")[-1]

                    elif "Secondary Status:" in text:
                        article_data["Secondary Status"] = text.split(": ")[-1]

                    elif "City:" in text:
                        article_data["City"] = li.find_element(By.XPATH, ".//span").text

                    elif "State:" in text:
                        article_data["State"] = li.find_element(By.XPATH, ".//span").text

                    elif "County:" in text:
                        article_data["County"] = text.split(": ")[-1]

                    elif "Zip:" in text:
                        article_data["Zip"] = text.split(": ")[-1]

                # Avoid duplicates in the results
                if article_data not in articles:
                    articles.append(article_data)

            except Exception as e:
                print(f"Error processing <ul> element: {e}")

        # Scroll down to load more results
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)

        # Check if new content has loaded by comparing page height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # Break if no new content is loaded
            break
        last_height = new_height

    # Step 8: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Print or save the DataFrame
    print(df)
    df.to_csv("active_sterile_pharmacy_licenses_california.csv", index=False)

finally:
    # Close the browser
    driver.quit()


## New Attempt

In [37]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the License Type dropdown to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Step 3: Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Step 4: Select "Sterile Compounding Pharmacy" from the License Type dropdown
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 5: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 6: Wait for the initial results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "actions"))
    )

    # Step 7: Loop through IDs and scrape data
    articles = []

    for article_id in range(845):  # IDs from 0 to 844
        try:
            # Construct the article selector dynamically using XPath
            article_selector = f"//article[@id='{article_id}']"
            
            # Locate the article element by XPath
            article_element = driver.find_element(By.XPATH, article_selector)
            
            # Locate the <ul> element with class "actions" within the article
            ul_element = article_element.find_element(By.CSS_SELECTOR, "ul.actions")
            
            # Initialize a dictionary to store data
            article_data = {}

            # Extract the pharmacy name (h3 within the <ul>)
            try:
                article_data["Pharmacy Name"] = ul_element.find_element(By.XPATH, "./li/h3").text
            except:
                article_data["Pharmacy Name"] = "Not Available"

            # Extract data for each relevant field
            for li in ul_element.find_elements(By.XPATH, "./li"):
                text = li.text.strip()

                if "License Number:" in text:
                    article_data["License Number"] = li.find_element(By.XPATH, ".//a/span").text

                elif "License Type:" in text:
                    article_data["License Type"] = text.split(": ")[-1]

                elif "License Status:" in text:
                    article_data["License Status"] = text.split(": ")[-1]

                elif "Expiration Date:" in text:
                    article_data["Expiration Date"] = text.split(": ")[-1]

                elif "Secondary Status:" in text:
                    article_data["Secondary Status"] = text.split(": ")[-1]

                elif "City:" in text:
                    article_data["City"] = li.find_element(By.XPATH, ".//span").text

                elif "State:" in text:
                    article_data["State"] = li.find_element(By.XPATH, ".//span").text

                elif "County:" in text:
                    article_data["County"] = text.split(": ")[-1]

                elif "Zip:" in text:
                    article_data["Zip"] = text.split(": ")[-1]

            # Avoid duplicates in the results
            if article_data not in articles:
                articles.append(article_data)

        except Exception as e:
            print(f"Error processing article with ID {article_id}: {e}")

    # At the end of the loop, `articles` will contain all the scraped data.
    print(f"Scraped {len(articles)} articles.")

    # Step 8: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt5.csv", index=False)

finally:
    # Close the browser
    driver.quit()


## Attempt 6

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters and set them
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 4: Wait for the first results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "post.yes"))
    )

    # Step 5: Scroll and Load More Results
    last_height = driver.execute_script("return document.body.scrollHeight")
    articles = []
    while True:
        # Find all articles
        new_articles = driver.find_elements(By.CSS_SELECTOR, "article.post.yes")
        articles.extend(new_articles)

        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Allow time for new content to load

        # Check if the page height has changed
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Step 6: Extract Information from Each Article
    results = []
    for article in articles:
        try:
            title = article.find_element(By.CSS_SELECTOR, "h3").text
            license_number = article.find_element(By.CSS_SELECTOR, "span[id^='lic']").text
            license_type = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Type:')]]").text
            license_status = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'License Status:')]]").text
            expiration_date = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Expiration Date:')]]").text
            city = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'City:')]]").text
            state = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'State:')]]").text
            county = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'County:')]]").text
            zip_code = article.find_element(By.XPATH, ".//li[strong[contains(text(), 'Zip:')]]").text

            results.append({
                "Title": title,
                "License Number": license_number,
                "License Type": license_type,
                "License Status": license_status,
                "Expiration Date": expiration_date,
                "City": city,
                "State": state,
                "County": county,
                "Zip": zip_code,
            })
        except Exception as e:
            print(f"Error processing article: {e}")

    # Step 7: Save to a DataFrame
    df = pd.DataFrame(results)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt6.csv", index=False)

finally:
    # Close the browser
    driver.quit()


##  Attempt 7:
Last attempt rendered the correct fields but gave duplicates for the first 20 values available, over and over, and ocne removed, we only had 0 through 19 accounted for.  

In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 4: Wait for the first results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "post.yes"))
    )

    # Step 5: Scroll and Load More Results
    # We'll iterate through article IDs instead of scrolling indefinitely to avoid repetition.
    articles = []
    for article_id in range(845):  # Assuming article IDs go from 0 to 844
        try:
            # Construct the XPath for each article dynamically using the article_id
            article_selector = f"//article[@id='{article_id}']"
            
            # Locate the article element using the dynamically generated XPath
            article_element = driver.find_element(By.XPATH, article_selector)
            
            # Extract the <ul> with class "actions" within the article
            ul_element = article_element.find_element(By.CSS_SELECTOR, "ul.actions")
            
            # Initialize a dictionary to store data for the current article
            article_data = {}

            # Extract information from the article's <ul> element
            try:
                article_data["Pharmacy Name"] = ul_element.find_element(By.XPATH, "./li/h3").text
            except:
                article_data["Pharmacy Name"] = "Not Available"

            for li in ul_element.find_elements(By.XPATH, "./li"):
                text = li.text.strip()

                # Extract specific information based on the li content
                if "License Number:" in text:
                    article_data["License Number"] = li.find_element(By.XPATH, ".//a/span").text
                elif "License Type:" in text:
                    article_data["License Type"] = text.split(": ")[-1]
                elif "License Status:" in text:
                    article_data["License Status"] = text.split(": ")[-1]
                elif "Expiration Date:" in text:
                    article_data["Expiration Date"] = text.split(": ")[-1]
                elif "Secondary Status:" in text:
                    article_data["Secondary Status"] = text.split(": ")[-1]
                elif "City:" in text:
                    article_data["City"] = li.find_element(By.XPATH, ".//span").text
                elif "State:" in text:
                    article_data["State"] = li.find_element(By.XPATH, ".//span").text
                elif "County:" in text:
                    article_data["County"] = text.split(": ")[-1]
                elif "Zip:" in text:
                    article_data["Zip"] = text.split(": ")[-1]

            # Append the article data to the list, avoiding duplicates
            if article_data not in articles:
                articles.append(article_data)

        except Exception as e:
            print(f"Error processing article with ID {article_id}: {e}")
            continue  # Skip to the next article if an error occurs

    # At the end of the loop, `articles` will contain all the scraped data.
    print(f"Scraped {len(articles)} articles.")

    # Step 6: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt7.csv", index=False)

finally:
    # Close the browser
    driver.quit()


## Attempt 8
Attempt 7 was a regression; go back and try again 

In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 4: Wait for the results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "post.yes"))
    )

    # Step 5: Scroll and Load More Results
    # Scroll through the page to load all articles before scraping
    last_height = driver.execute_script("return document.body.scrollHeight")
    articles = []

    while True:
        # Find all article elements that match the criteria (using CSS class 'post.yes')
        new_articles = driver.find_elements(By.CSS_SELECTOR, "article.post.yes")
        articles.extend(new_articles)

        # Scroll down the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for the new content to load

        # Check if the page height has changed after scrolling. If no new height, stop scrolling.
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Step 6: Extract Information from Each Article
    results = []
    for article in articles:
        try:
            # Get the ul with class "actions" to extract data from the article
            ul_element = article.find_element(By.CSS_SELECTOR, "ul.actions")
            article_data = {}

            # Extract Pharmacy Name
            try:
                article_data["Pharmacy Name"] = ul_element.find_element(By.XPATH, "./li/h3").text
            except Exception as e:
                article_data["Pharmacy Name"] = "Not Available"

            # Loop through each <li> in the <ul> and extract relevant data
            for li in ul_element.find_elements(By.XPATH, "./li"):
                text = li.text.strip()

                # Extract specific data based on the li's content
                if "License Number:" in text:
                    article_data["License Number"] = li.find_element(By.XPATH, ".//a/span").text
                elif "License Type:" in text:
                    article_data["License Type"] = text.split(": ")[-1]
                elif "License Status:" in text:
                    article_data["License Status"] = text.split(": ")[-1]
                elif "Expiration Date:" in text:
                    article_data["Expiration Date"] = text.split(": ")[-1]
                elif "Secondary Status:" in text:
                    article_data["Secondary Status"] = text.split(": ")[-1]
                elif "City:" in text:
                    article_data["City"] = li.find_element(By.XPATH, ".//span").text
                elif "State:" in text:
                    article_data["State"] = li.find_element(By.XPATH, ".//span").text
                elif "County:" in text:
                    article_data["County"] = text.split(": ")[-1]
                elif "Zip:" in text:
                    article_data["Zip"] = text.split(": ")[-1]

            # Append article data to results list
            if article_data not in results:
                results.append(article_data)

        except Exception as e:
            print(f"Error processing article: {e}")
            continue  # Skip to the next article if there's an error

    # At the end of the loop, `results` will contain all the scraped data
    print(f"Scraped {len(results)} articles.")

    # Step 7: Save the results to a DataFrame
    df = pd.DataFrame(results)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt8.csv", index=False)

finally:
    # Close the browser
    driver.quit()


## Attempt 9:
Explicitly try loading all values first, and then return and loop through them.  

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 4: Scroll to the bottom of the page to load all articles
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll to load all articles
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for the new content to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Step 5: Loop through all articles by ID (from 0 to 844)
    results = []
    for article_id in range(845):  # ID range from 0 to 844
        try:
            # Construct the article selector dynamically using XPath
            article_selector = f"//article[@id='{article_id}']"
            
            # Locate the article element by XPath
            article_element = driver.find_element(By.XPATH, article_selector)
            
            # Locate the <ul> element with class "actions" within the article
            ul_element = article_element.find_element(By.CSS_SELECTOR, "ul.actions")
            
            # Initialize a dictionary to store data
            article_data = {}

            # Extract the pharmacy name (h3 within the <ul>)
            try:
                article_data["Pharmacy Name"] = ul_element.find_element(By.XPATH, "./li/h3").text
            except:
                article_data["Pharmacy Name"] = "Not Available"

            # Extract data for each relevant field
            for li in ul_element.find_elements(By.XPATH, "./li"):
                text = li.text.strip()

                if "License Number:" in text:
                    article_data["License Number"] = li.find_element(By.XPATH, ".//a/span").text

                elif "License Type:" in text:
                    article_data["License Type"] = text.split(": ")[-1]

                elif "License Status:" in text:
                    article_data["License Status"] = text.split(": ")[-1]

                elif "Expiration Date:" in text:
                    article_data["Expiration Date"] = text.split(": ")[-1]

                elif "Secondary Status:" in text:
                    article_data["Secondary Status"] = text.split(": ")[-1]

                elif "City:" in text:
                    article_data["City"] = li.find_element(By.XPATH, ".//span").text

                elif "State:" in text:
                    article_data["State"] = li.find_element(By.XPATH, ".//span").text

                elif "County:" in text:
                    article_data["County"] = text.split(": ")[-1]

                elif "Zip:" in text:
                    article_data["Zip"] = text.split(": ")[-1]

            # Avoid duplicates in the results
            if article_data not in results:
                results.append(article_data)

        except Exception as e:
            print(f"Error processing article with ID {article_id}: {e}")

    # At the end of the loop, `results` will contain all the scraped data
    print(f"Scraped {len(results)} articles.")

    # Step 6: Save the results to a DataFrame
    df = pd.DataFrame(results)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt9.csv", index=False)

finally:
    # Close the browser
    driver.quit()


## Attempt 10 
Last attempt generated almost all of the pharmacies, but title only.  
Continue trying to pick up all of the items on the page.  

In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 4: Implement infinite scrolling
    last_height = driver.execute_script("return document.body.scrollHeight")
    articles = []

    while True:
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new content to load
        time.sleep(2)

        # Get new height
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        last_height = new_height

        # Extract data from all articles on the page
        article_elements = driver.find_elements(By.CSS_SELECTOR, ".post.yes")
        
        for element in article_elements:
            try:
                article_data = {}

                # Extract pharmacy name (h3 within the <ul>)
                h3_element = element.find_element(By.XPATH, ".//li/h3")
                article_data["Pharmacy Name"] = h3_element.text.strip()

                # Extract other data fields
                for li in element.find_elements(By.CSS_SELECTOR, ".actions li"):
                    text = li.text.strip()
                    key_value = text.split(": ", 1)
                    if len(key_value) == 2:
                        key, value = key_value
                        article_data[key] = value.strip()

                articles.append(article_data)
            except Exception as e:
                print(f"Error processing article: {e}")

    # Step 5: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt10.csv", index=False)

finally:
    # Close the browser
    driver.quit()


## Attempt 11
Last one got all the items but looped repeatedly through the first 20 or so results, we still need to capture the totality of the results on the page. 

In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 4: Implement infinite scrolling
    last_height = driver.execute_script("return document.body.scrollHeight")
    articles = []

    while True:
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new content to load
        time.sleep(2)

        # Get new height
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        last_height = new_height

        # Extract data from all articles on the page
        article_elements = driver.find_elements(By.CSS_SELECTOR, ".post.yes")
        
        for element in article_elements:
            try:
                article_data = {}

                # Extract pharmacy name (h3 within the <ul>)
                h3_element = element.find_element(By.XPATH, ".//li/h3")
                article_data["Pharmacy Name"] = h3_element.text.strip()

                # Extract other data fields
                for li in element.find_elements(By.CSS_SELECTOR, ".actions li"):
                    text = li.text.strip()
                    key_value = text.split(": ", 1)
                    if len(key_value) == 2:
                        key, value = key_value
                        article_data[key] = value.strip()

                articles.append(article_data)
            except Exception as e:
                print(f"Error processing article: {e}")

    # Step 5: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt11.csv", index=False)

finally:
    # Close the browser
    driver.quit()


## Attempt 12
Still having issues with the data repeating, we will try again to grab beyond the first 20 results.  

In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 4: Extract data from all ul elements with class "actions"
    actions_elements = driver.find_elements(By.CSS_SELECTOR, ".post.yes ul.actions")

    articles = []

    for element in actions_elements:
        try:
            article_data = {}

            # Extract pharmacy name (h3 within the <ul>)
            h3_element = element.find_element(By.XPATH, ".//li/h3")
            article_data["Pharmacy Name"] = h3_element.text.strip()

            # Extract other data fields
            for li in element.find_elements(By.CSS_SELECTOR, ".actions li"):
                text = li.text.strip()
                key_value = text.split(": ", 1)
                if len(key_value) == 2:
                    key, value = key_value
                    article_data[key] = value.strip()

            articles.append(article_data)
        except Exception as e:
            print(f"Error processing article: {e}")

    # Step 5: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt12.csv", index=False)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the browser
    driver.quit()


## Attempt 13
Still trying - only got first 20 records again 

In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Implement a more robust infinite scrolling mechanism
    SCROLL_PAUSE_TIME = 5

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

        # Extract data after each scroll
        actions_elements = driver.find_elements(By.CSS_SELECTOR, ".post.yes ul.actions")
        
        for element in actions_elements:
            try:
                article_data = {}

                # Extract pharmacy name (h3 within the <ul>)
                h3_element = element.find_elements(By.CSS_SELECTOR, "li > h3")
                if h3_element:
                    article_data["Pharmacy Name"] = h3_element[0].text.strip()

                # Extract other data fields
                for li in element.find_elements(By.CSS_SELECTOR, ".actions li"):
                    text = li.text.strip()
                    key_value = text.split(": ", 1)
                    if len(key_value) == 2:
                        key, value = key_value
                        article_data[key] = value.strip()

                articles.append(article_data)
            except Exception as e:
                print(f"Error processing article: {e}")

    # Step 5: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies.csv", index=False)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the browser
    driver.quit()


## Attempt 14
If the above fails I will try this which was combined from the results that got most of the names with the one that got all the fields for the first 20 items 

In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Implement a more robust infinite scrolling mechanism
    SCROLL_PAUSE_TIME = 5

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Step 4: Extract data from all ul elements with class "actions"
    actions_elements = driver.find_elements(By.CSS_SELECTOR, ".post.yes ul.actions")
    
    articles = []

    for element in actions_elements:
        try:
            article_data = {}

            # Extract pharmacy name (h3 within the <ul>)
            h3_element = element.find_elements(By.CSS_SELECTOR, "li > h3")
            if h3_element:
                article_data["Pharmacy Name"] = h3_element[0].text.strip()

            # Extract other data fields
            for li in element.find_elements(By.CSS_SELECTOR, ".actions li"):
                text = li.text.strip()
                key_value = text.split(": ", 1)
                if len(key_value) == 2:
                    key, value = key_value
                    article_data[key] = value.strip()

            articles.append(article_data)
        except Exception as e:
            print(f"Error processing article: {e}")

    # Step 5: Save the results to a DataFrame
    df = pd.DataFrame(articles)
    
    #Drop Rows that are all NaN
    df = df.dropna()

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt14.csv", index=False)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the browser
    driver.quit()


## Attempt 15
Previous attempts all continue to stall out after 20 pharmacies; I have yet to be able to capture all of the pharmacies on the page.  

In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

def extract_data():
    """Helper function to extract pharmacy data"""
    actions_elements = driver.find_elements(By.CSS_SELECTOR, ".post.yes ul.actions")
    articles = []
    
    for element in actions_elements:
        try:
            article_data = {}

            # Extract pharmacy name (h3 within the <ul>)
            h3_element = element.find_elements(By.CSS_SELECTOR, "li > h3")
            if h3_element:
                article_data["Pharmacy Name"] = h3_element[0].text.strip()

            # Extract other data fields
            for li in element.find_elements(By.CSS_SELECTOR, ".actions li"):
                text = li.text.strip()
                key_value = text.split(": ", 1)
                if len(key_value) == 2:
                    key, value = key_value
                    article_data[key] = value.strip()

            articles.append(article_data)
        except Exception as e:
            print(f"Error processing article: {e}")
    
    return articles

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Implement scrolling to grab records in chunks of 20
    SCROLL_PAUSE_TIME = 5
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Container for all the data
    all_articles = []
    
    while True:
        # Extract first 20 records (or fewer if there are no more records)
        articles = extract_data()
        all_articles.extend(articles)
        
        # Scroll down to bottom to load more data
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # No new content loaded, break the loop
            break
        last_height = new_height

    # Step 4: Save the results to a DataFrame
    df = pd.DataFrame(all_articles)

    # Drop rows that are all NaN
    df = df.dropna()

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt15.csv", index=False)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the browser
    driver.quit()


## Attempt 16:  Combine Selenium & Beautiful Soup
If the above does not work I will try to use Selenium to return the source HTML and then use beautifulsoup to parse the html properly.  

In [50]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

def scroll_to_bottom():
    """Scroll the entire page to load all content"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    SCROLL_PAUSE_TIME = 3  # Adjust this based on how long the page takes to load
    
    while True:
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate the new scroll height and compare it to the previous height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Exit loop when no more content is loaded
        last_height = new_height

def extract_data_from_html(html):
    """Extract data from the given HTML using BeautifulSoup"""
    soup = BeautifulSoup(html, 'html.parser')
    articles = []

    # Extract pharmacy data from the soup object
    actions_elements = soup.select(".post.yes ul.actions")
    
    for element in actions_elements:
        try:
            article_data = {}

            # Extract pharmacy name (h3 within the <ul>)
            h3_element = element.select("li > h3")
            if h3_element:
                article_data["Pharmacy Name"] = h3_element[0].text.strip()

            # Extract other data fields
            for li in element.select(".actions li"):
                text = li.text.strip()
                key_value = text.split(": ", 1)
                if len(key_value) == 2:
                    key, value = key_value
                    article_data[key] = value.strip()

            articles.append(article_data)
        except Exception as e:
            print(f"Error processing article: {e}")
    
    return articles

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 4: Scroll the entire page to load all content
    scroll_to_bottom()

    # Step 5: Get the entire page source after scrolling
    html = driver.page_source

    # Step 6: Extract data from the HTML source using BeautifulSoup
    articles = extract_data_from_html(html)

    # Step 7: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Drop rows that are all NaN
    df = df.dropna()

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt_with_soup.csv", index=False)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the browser
    driver.quit()


## Attempt 17: Soup Part 2
Last attempt with soup did return a table with ... 20 responses.  Still need to figure out how we can get all of them! 

In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

def scroll_to_bottom_until_end_text():
    """Scroll the entire page to load all content until the end text is visible."""
    SCROLL_PAUSE_TIME = 3  # Adjust this based on how long the page takes to load
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Check if the "You have reached the end of your results" text is present
        try:
            end_text = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'You have reached the end of your results')]"))
            )
            if end_text:
                break
        except:
            # If the end text isn't found, we continue scrolling
            pass
        
        # Calculate the new scroll height and compare it to the previous height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Exit loop when no more content is loaded
        last_height = new_height

def extract_data_from_html(html):
    """Extract data from the given HTML using BeautifulSoup."""
    soup = BeautifulSoup(html, 'html.parser')
    articles = []

    # Extract pharmacy data from the soup object
    actions_elements = soup.select(".post.yes ul.actions")
    
    for element in actions_elements:
        try:
            article_data = {}

            # Extract pharmacy name (h3 within the <ul>)
            h3_element = element.select("li > h3")
            if h3_element:
                article_data["Pharmacy Name"] = h3_element[0].text.strip()

            # Extract other data fields
            for li in element.select(".actions li"):
                text = li.text.strip()
                key_value = text.split(": ", 1)
                if len(key_value) == 2:
                    key, value = key_value
                    article_data[key] = value.strip()

            articles.append(article_data)
        except Exception as e:
            print(f"Error processing article: {e}")
    
    return articles

try:
    # Step 1: Open the webpage
    url = "https://search.dca.ca.gov/advanced"
    driver.get(url)

    # Step 2: Wait for the filters to load and set them (Primary Status and License Type)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "primaryStatusCodes"))
    )

    # Set the Primary Status to "Active"
    status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
    status_dropdown.select_by_visible_text("Active")

    # Set the License Type to "Sterile Compounding Pharmacy"
    license_dropdown = Select(driver.find_element(By.ID, "licenseType"))
    license_dropdown.select_by_visible_text("Sterile Compounding Pharmacy")

    # Step 3: Wait for the "Search" button and click it
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "srchSubmitHome"))
    )
    search_button = driver.find_element(By.ID, "srchSubmitHome")
    search_button.click()

    # Step 4: Scroll the entire page to load all content until we reach the "end of results" text
    scroll_to_bottom_until_end_text()

    # Step 5: Get the entire page source after scrolling
    html = driver.page_source

    # Step 6: Extract data from the HTML source using BeautifulSoup
    articles = extract_data_from_html(html)

    # Step 7: Save the results to a DataFrame
    df = pd.DataFrame(articles)

    # Drop rows that are all NaN
    df = df.dropna()

    # Print or save the DataFrame
    print(df)
    df.to_csv("pharmacies_attempt_with_end_text.csv", index=False)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the browser
    driver.quit()


In [53]:
html

## Attempt 18 with selenium and soup 

In [58]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1. Navigate to the URL
driver.get("https://search.dca.ca.gov/advanced")

# 2. Wait until you see the ID "srchSubmitHome"
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "srchSubmitHome")))

# 3. Select "Active" from the dropdown by ID "primaryStatusCodes"
status_dropdown = driver.find_element(By.ID, "primaryStatusCodes")
status_dropdown.click()
status_dropdown.find_element(By.XPATH, "//option[text()='Active']").click()

# 4. Enter "Sterile Compounding Pharmacy" in the licenseType field
license_type_field = driver.find_element(By.ID, "licenseType")
license_type_field.send_keys("Sterile Compounding Pharmacy")

# 5. Click the search button by ID "srchSubmitHome"
search_button = driver.find_element(By.ID, "srchSubmitHome")
search_button.click()

# 6. Allow the page to load and scroll until "You have reached the end of your results"
time.sleep(5)  # Let the page load

# Scroll to the bottom until the "You have reached the end of your results" text is found
while True:
    page_end_element = driver.find_elements(By.XPATH, "//*[text()='You have reached the end of your results']")
    if page_end_element:
        break
    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 7. Get the entire HTML of the page
html = driver.page_source

# 8. Parse the HTML with BeautifulSoup and extract the data
soup = BeautifulSoup(html, "html.parser")
articles = []

# Find all article tags with the ul tag having class "actions"
for article in soup.find_all("article"):
    actions_element = article.find("ul", class_="actions")
    if actions_element:
        article_data = {}

        # Extract pharmacy name (h3 within the <ul>)
        h3_element = actions_element.find("h3")
        if h3_element:
            article_data["Pharmacy Name"] = h3_element.text.strip()

        # Extract other data fields
        for li in actions_element.find_all("li"):
            text = li.text.strip()
            key_value = text.split(": ", 1)
            if len(key_value) == 2:
                key, value = key_value
                article_data[key] = value.strip()

        articles.append(article_data)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(articles)

# Save the DataFrame to a CSV file
df.to_csv("attempt_18.csv", index=False)

# Optionally print a message confirming that the CSV was saved
print("CSV file 'attempt_18.csv' has been saved.")

# Close the browser
driver.quit()


## Attempt 19
THe previous works minus the filter for active, this attempts to fix that.  

In [60]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.support.ui import Select  # Import Select for dropdowns

# Set up Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1. Navigate to the URL
driver.get("https://search.dca.ca.gov/advanced")

# 2. Wait until you see the ID "srchSubmitHome"
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "srchSubmitHome")))

# 3. Set the Primary Status to "Active"
status_dropdown = Select(driver.find_element(By.ID, "primaryStatusCodes"))
status_dropdown.select_by_visible_text("Active")

# 4. Enter "Sterile Compounding Pharmacy" in the licenseType field
license_type_field = driver.find_element(By.ID, "licenseType")
license_type_field.send_keys("Sterile Compounding Pharmacy")

# 5. Click the search button by ID "srchSubmitHome"
search_button = driver.find_element(By.ID, "srchSubmitHome")
search_button.click()

# 6. Allow the page to load and scroll until "You have reached the end of your results"
time.sleep(5)  # Let the page load

# Scroll to the bottom until the "You have reached the end of your results" text is found
while True:
    page_end_element = driver.find_elements(By.XPATH, "//*[text()='You have reached the end of your results']")
    if page_end_element:
        break
    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 7. Get the entire HTML of the page
html = driver.page_source

# 8. Parse the HTML with BeautifulSoup and extract the data
soup = BeautifulSoup(html, "html.parser")
articles = []

# Find all article tags with the ul tag having class "actions"
for article in soup.find_all("article"):
    actions_element = article.find("ul", class_="actions")
    if actions_element:
        article_data = {}

        # Extract pharmacy name (h3 within the <ul>)
        h3_element = actions_element.find("h3")
        if h3_element:
            article_data["Pharmacy Name"] = h3_element.text.strip()

        # Extract other data fields
        for li in actions_element.find_all("li"):
            text = li.text.strip()
            key_value = text.split(": ", 1)
            if len(key_value) == 2:
                key, value = key_value
                article_data[key] = value.strip()

        articles.append(article_data)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(articles)

# Save the DataFrame to a CSV file
df.to_csv("attempt_19.csv", index=False)

# Optionally print a message confirming that the CSV was saved
print("CSV file 'attempt_19.csv' has been saved.")

# Close the browser
driver.quit()
